# Notes on Rust

## Make Jupyter knows Rust

Running this notebook requires Rust's jupyter kernel installed through `cargo` build chain which requires the Rust ecosystem.
- install system dependency `sudo pamac install cmake` 
- install the jupyter-kernel `cargo install evcxr_jupyter`
  ```
    Updating crates.io index
    Installing evcxr_jupyter v0.7.0
     Compiling libc v0.2.81
     ...
     Compiling evcxr_jupyter v0.7.0
     Finished release [optimized] target(s) in 3m 46s
    Installing /home/tilo/.cargo/bin/evcxr_jupyter
     Installed package `evcxr_jupyter v0.7.0` (executable `evcxr_jupyter`)

- activate jupyter-kernle ` ~/.cargo/bin/evcxr_jupyter --install` 
  ```
    Writing /home/tilo/.local/share/jupyter/kernels/rust/kernel.json
    Writing /home/tilo/.local/share/jupyter/kernels/rust/logo-32x32.png
    Writing /home/tilo/.local/share/jupyter/kernels/rust/logo-64x64.png
    Writing /home/tilo/.local/share/jupyter/kernels/rust/logo-LICENSE.md
    Installation complete


In [2]:
const msg: &str = "Hello EvCxR";

In [3]:
msg

"Hello EvCxR"

## Display rustc version

Followed [this advise: How can a Rust program access metadata from its Cargo package?](https://stackoverflow.com/questions/27840394/how-can-a-rust-program-access-metadata-from-its-cargo-package) and found function in module `std::env`.

In [4]:
std::env!("CARGO_MANIFEST_DIR", "none")

"/tmp/.tmpe62nAM"

In [5]:
std::env!("CARGO_PKG_NAME", "none")

"ctx"

In [6]:
let env_vars  = vec!("CARGO_MANIFEST_DIR","CARGO_PKG_AUTHORS","CARGO_PKG_DESCRIPTION","CARGO_PKG_HOMEPAGE","CARGO_PKG_NAME","CARGO_PKG_REPOSITORY","CARGO_PKG_VERSION","CARGO_PKG_VERSION_MAJOR","CARGO_PKG_VERSION_MINOR","CARGO_PKG_VERSION_PATCH","CARGO_PKG_VERSION_PRE");
env_vars[0]

"CARGO_MANIFEST_DIR"

In [7]:
for var in env_vars {println!("{} : {}", var, std::env!(var, "none"))}

Error: expected string literal

Don't know how to solve this for now. From type point of view both are string slices, the literal as well as array member.

In [8]:
std::any::Any::type_id(env_vars[0])

TypeId { t: 12392834530613204498 }

In [9]:
std::any::Any::type_id("CARGO_MANIFEST_DIR")

TypeId { t: 12392834530613204498 }

Let's resolve type's name.

In [10]:
std::any::type_name_of_val(env_vars[0])

Error: use of unstable library feature 'type_name_of_val'

In [11]:
:toolchain nightly

Toolchain: nightly


In [12]:
#![feature(type_name_of_val)]
let nightly_env_vars  = vec!("CARGO_MANIFEST_DIR","CARGO_PKG_AUTHORS","CARGO_PKG_DESCRIPTION","CARGO_PKG_HOMEPAGE","CARGO_PKG_NAME","CARGO_PKG_REPOSITORY","CARGO_PKG_VERSION","CARGO_PKG_VERSION_MAJOR","CARGO_PKG_VERSION_MINOR","CARGO_PKG_VERSION_PATCH","CARGO_PKG_VERSION_PRE");
std::any::type_name_of_val(nightly_env_vars[0])

The type of the variable env_vars was redefined, so was lost.


"str"

In [13]:
#![feature(type_name_of_val)]
std::any::type_name_of_val("CARGO_MANIFEST_DIR")

"str"

## Plotting with Plotters

A copy for evaluation from [Data Analysis with Rust Notebooks](https://datacrayon.com/posts/programming/rust-notebooks/plotting-with-plotters/) by Data Crayon.

See also:
- [Crate plotters - A Rust drawing library focus on data plotting for both WASM and native applications](https://docs.rs/plotters/latest/plotters/index.html)
- [Developers Guide](https://plotters-rs.github.io/book/)

Plotters: "Because `evcxr` uses only SVG images and all types of series, so we don't need other types of backend. So we should put `default_features = false, features = ["evcxr", "all_series"]`" when dependency `plotters` is to be loaded into `evcxr`.

Function [`evcxr_figure()`](https://github.com/38/plotters/blob/26b24b1d30521af98e957a3a907d76b038e66c54/src/evcxr.rs#L31) is the integration provided by `plotters` with `evcxr` and is compiled in when [feature `evcxr` is configured](https://github.com/38/plotters/blob/26b24b1d30521af98e957a3a907d76b038e66c54/src/lib.rs#L748). It has following signature: 

> `pub fn plotters::evcxr::evcxr_figure<
     Draw: FnOnce(plotters::drawing::DrawingArea<
                    plotters::prelude::SVGBackend, 
                    plotters::coord::Shift>
                 ) -> Result<
                        (),
                        Box<dyn std::error::Error>>
    >(size: (u32, u32), draw: Draw) -> plotters::evcxr::SVGWrapper`. 

`evcxr_figure()` takes as arguments the size of the a drawing area and a [closure expression](https://doc.rust-lang.org/reference/expressions/closure-expr.html) creating an annonymous type implementing trait [`FnOnce`](https://doc.rust-lang.org/std/ops/trait.FnOnce.html) with it's method `call_once()` performing the call operation. `evcxr_figure()` returns an object of type `SVGWrapper` which implements following methods:
- `pub fn evcxr_display(&self)`,
- `pub fn style<S: Into<String>>(mut self, style: S) -> Self`

and which implements finally the `std::fmt::Debug` trait according to the convention described by [`evcxr_jupyter` for custom output](https://github.com/google/evcxr/blob/38981aeccd20f9bb1bad65cd6d47ed8898caa2a6/evcxr_jupyter/README.md#custom-output). 

In the function to create a figure to be displayed by `evcxr_jupyter` following steps are taken:
1. [The abstraction of a drawing area](https://docs.rs/plotters/latest/plotters/drawing/struct.DrawingArea.html) is hand over as variable `root` and with help of it's method [filled](https://docs.rs/plotters/latest/plotters/drawing/struct.DrawingArea.html#method.fill) in color [white](https://docs.rs/plotters/latest/plotters/style/colors/constant.WHITE.html).
2. A [helper object to create a chart context](https://docs.rs/plotters/latest/plotters/chart/struct.ChartBuilder.html) is created with its method [on](https://docs.rs/plotters/latest/plotters/chart/struct.ChartBuilder.html#method.on) returning the chart builder followed by chain of method calls:
  - [The caption of the chart with a text style](https://docs.rs/plotters/latest/plotters/chart/struct.ChartBuilder.html#method.caption) is set returning the chart builder instance.
  - [The margin size of the chart](https://docs.rs/plotters/latest/plotters/chart/struct.ChartBuilder.html#method.margin) is set returning the chart builder instance.
  - [The size of X label area](https://docs.rs/plotters/latest/plotters/chart/struct.ChartBuilder.html#method.x_label_area_size) is set returning the chart builder instance.
  - [The size of the Y label area](https://docs.rs/plotters/latest/plotters/chart/struct.ChartBuilder.html#method.y_label_area_size) is set returning the chart builder instance.
  - And finally [build the chart with a 2D Cartesian coordinate system.](https://docs.rs/plotters/latest/plotters/chart/struct.ChartBuilder.html#method.build_cartesian_2d) The function will return a `Result` object with either a [ChartContext<'a, DB, Cartesian2d<X::CoordDescType, Y::CoordDescType>](https://docs.rs/plotters/latest/plotters/chart/struct.ChartContext.html), where a data series can be rendered on, or a `DrawingAreaErrorKind<DB::ErrorType>`.
3. The [chart context](https://docs.rs/plotters/latest/plotters/chart/struct.ChartContext.html) method [**configure_mesh()**](https://docs.rs/plotters/latest/plotters/chart/struct.ChartContext.html#method.configure_mesh) initializes a mesh configuration object and returning a [MeshStyle](https://docs.rs/plotters/latest/plotters/chart/struct.MeshStyle.html). The mesh drawing can be finalized by calling the MeshStyle's method [draw](https://docs.rs/plotters/latest/plotters/chart/struct.MeshStyle.html#method.draw) which draws the configured mesh on the target plot returning a `Result` object ( either `()` or `DrawingAreaErrorKind<DB::ErrorType>`).
4. The [chart context's](https://docs.rs/plotters/latest/plotters/chart/struct.ChartContext.html) method [**draw_series()**](https://docs.rs/plotters/latest/plotters/chart/struct.ChartContext.html#method.draw_series) draws a data series returning a `Result` object with either a [series' annotation](https://docs.rs/plotters/latest/plotters/chart/struct.SeriesAnno.html) or a `DrawingAreaErrorKind<DB::ErrorType>` followed by a chain of method calls. A data series in Plotters is abstracted as an iterator of elements.
  - The [line series](https://docs.rs/plotters/latest/plotters/series/struct.LineSeries.html) object, which takes an iterator of points in guest coordinate system with a ShapeStyle and creates the element rendering the line plot.
  - [The label](https://docs.rs/plotters/latest/plotters/chart/struct.SeriesAnno.html#method.label) sets the string used as a label for current series.
  - [The legend](https://docs.rs/plotters/latest/plotters/chart/struct.SeriesAnno.html#method.legend) sets the element creator function. *Note*: The creation function uses a shifted pixel-based coordinate system. And place the point (0,0) to the mid-right point of the shape.
5. The [chart context's](https://docs.rs/plotters/latest/plotters/chart/struct.ChartContext.html) method [**configure_series_labels()**](https://docs.rs/plotters/latest/plotters/chart/struct.ChartContext.html#method.configure_series_labels) configures the styles for drawing series labels in the chart and returns a [SeriesLabelStyle](https://docs.rs/plotters/latest/plotters/chart/struct.SeriesLabelStyle.html). By calling the SeriesLabelStyle's method [draw](https://docs.rs/plotters/latest/plotters/chart/struct.SeriesLabelStyle.html#method.draw) which draws the series label area returning a `Result` object ( either `()` or `DrawingAreaErrorKind<DB::ErrorType>`).
6. The closure "closes" with returning an empty tuple, the [`()` type](https://doc.rust-lang.org/std/primitive.unit.html) wrapped in a [ok variant](https://doc.rust-lang.org/std/result/enum.Result.html#variant.Ok) of a `Result` object.

In [2]:
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "line_series"] }
extern crate plotters;
use plotters::prelude::*;
let figure = plotters::evcxr::evcxr_figure((640, 480), |root| {
    root.fill(&plotters::style::colors::WHITE)?; // `?` required to overcome warning: "unused `Result` that must be used"
    let mut chart = plotters::chart::ChartBuilder::on(&root)
        .caption("y=x^2", ("Arial", 50).into_font())
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(-1f32..1f32, -0.1f32..1f32)?;

    chart.configure_mesh().draw()?;

    chart.draw_series(
        plotters::series::LineSeries::new(
            (-50..=50).map(|x| x as f32 / 50.0).map(|x| (x, x * x)),
            &plotters::style::colors::RED,)
        ).unwrap()
        .label("y = x^2")
        .legend(|(x,y)| 
            plotters::element::PathElement::new(
                vec![(x,y), (x + 20,y)],
                &plotters::style::colors::RED)
               );

    chart.configure_series_labels()
        .background_style(&plotters::style::colors::WHITE.mix(0.8))
        .border_style(&plotters::style::colors::BLACK)
        .draw()?;
    Ok(())
});
figure

y=x^2
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
0.2
 
 
 
0.4
 
 
 
0.6
 
 
 
0.8
 
 
 
 
-0.8
 
 
 
-0.6
 
 
 
-0.4
 
 
 
-0.2
 
 
 
0.0
 
 
 
0.2
 
 
 
0.4
 
 
 
0.6
 
 
 
0.8
 
 
 
1.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,56 41,69 47,82 53,96 59,109 65,122 71,134 77,147 83,159 89,170 95,182 101,193 107,204 113,214 119,225 125,235 131,245 137,254 143,263 149,272 155,281 161,289 167,297 173,305 179,313 185,320 191,327 197,333 203,340 209,346 215,352 221,357 227,362 233,367 239,372 245,376 251,380 257,384 263,388 269,391 275,394 281,397 287,399 293,401 299,403 305,405 311,406 317,407 323,408 329,408 335,408 341,408 347,408 353,407 359,406 365,405 371,403 377,401 383,399 389,397 395,394 401,391 407,388 413,384 419,380 425,376 431,372 437,367 443,362 449,357 455,352 461,346 467,340 473,333 479,327 485,320 491,313 497,305 503,297 509,289 515,281 521,272 527,263 533,254 539,245 545,235 551,225 557,214 563,204 569,193 575,182 581,170 587,159 593,147 599,134 605,122 611,109 617,96 623,82 629,69 635,56 "/>
 
 
 
y = x^2

Here is the example of `evcxr_jupyter` for cutom output which demonstrates the convention described to draw `figure` above and to "draw" `m` below.

In [15]:
use std::fmt::Debug;
pub struct Matrix<T> {pub values: Vec<T>, pub row_size: usize}
impl<T: Debug> Matrix<T> {
    pub fn evcxr_display(&self) {
        let mut html = String::new();
        html.push_str("<table>");
        for r in 0..(self.values.len() / self.row_size) {
            html.push_str("<tr>");
            for c in 0..self.row_size {
                html.push_str("<td>");
                html.push_str(&format!("{:?}", self.values[r * self.row_size + c]));
                html.push_str("</td>");
            }
            html.push_str("</tr>");
        }
        html.push_str("</table>");
        println!("EVCXR_BEGIN_CONTENT text/html\n{}\nEVCXR_END_CONTENT", html);
    }
}
let m = Matrix {values: vec![1,2,3,4,5,6,7,8,9], row_size: 3};
m

1,2,3
4,5,6
7,8,9


## Plotting with Plotly

Plotly has a [jupyter notebook example](https://github.com/igiagkiozis/plotly/blob/b8144ad116aa8aae2d1a626bed18ec008d1f283e/plotly/examples/jupyter_notebook_examples.ipynb)

In [19]:
:dep plotly = { version = ">=0.7.0" }
:dep itertools-num = "0.1.3"

In [20]:
extern crate plotly;
extern crate rand_distr;
extern crate itertools_num;
extern crate itertools;

In [22]:
use itertools_num::linspace;
use plotly::common::{
    ColorScale, ColorScalePalette, DashType, Fill, Font, Line, LineShape, Marker, Mode, Title,
};
use plotly::layout::{Axis, BarMode, Layout, Legend, TicksDirection};
use plotly::{Bar, NamedColor, Plot, Rgb, Rgba, Scatter};
use rand_distr::{Distribution, Normal, Uniform};

In [23]:
let n: usize = 100;
let t = vec![1,2,3,4,5,6,7,8,9,10];
let y = t.iter().map(|v| (*v as f64).powf(2.0)).collect::<Vec<f64>>();
let trace = Scatter::new(t,y).mode(Mode::Markers);
let mut plot = Plot::new();
plot.add_trace(trace);
let layout = Layout::new().height(525);
plot.set_layout(layout);
plot.notebook_display();

The type of the variable m was redefined, so was lost.
The type of the variable figure was redefined, so was lost.
